In [1]:
# 전체 코드
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly as py
import plotly.graph_objs as go
import requests
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.utils import *
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

names = ['feat_{}'.format(i) for i in range(73)]
df = pd.read_csv('ozone.data', names=names)
# df.dtypes  # object
df = df.apply(pd.to_numeric, errors='coerce')  # object를 숫자로 바꾼다. coerce 옵션은 숫자 변환 오류 시 NaN으로 변환
# df.dtypes  # float64
df.dropna(inplace=True)  # NaN 삭제(drop)
# df.describe()
Y = df['feat_72']
Y = to_categorical(Y)  # one hot encoding
df.drop(['feat_72'], axis=1, inplace=True)  # axis=1은 열, inplace=True는 자신의 정보 수정
X_train = np.array(df[:-100].values.tolist(), dtype=np.float64)
X_test = np.array(df[-100:].values.tolist(), dtype=np.float64)
Y_train = Y[:-100]
Y_test = Y[-100:]
print(X_test.shape)  # (100, 72)
print(X_train.shape) # (1747, 72)

# 훈련 숫자를 짝수로 맞춘다.
X_train = X_train[:1700]
Y_train = Y_train[:1700]
X_train = X_train.reshape(-1, 10, 72)
print(X_train.shape) # (170, 10, 72)
Y_train = Y_train.reshape(-1, 10, 2)
print(Y_train.shape) # (170, 10, 2)

model = Sequential()
model.add(LSTM(128, input_shape=(10, 72), return_sequences=True))
model.add(LSTM(256, return_sequences=True))  # Many to Many 방식
model.add(Dense(2, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
model.summary()

model.fit(X_train, Y_train, epochs=10, batch_size=1, validation_split=0.1)
# batch_size=1 옵션은 하나씩 전체 테스트

X_test = X_test.reshape(-1, 10, 72)
Y_test = Y_test.reshape(-1, 10, 2)

score = model.evaluate(X_test, Y_test)
print(score) # [0.16550002992153168, 0.98000001907348633]

Using TensorFlow backend.


(100, 72)
(1747, 72)
(170, 10, 72)
(170, 10, 2)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 128)           102912    
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 256)           394240    
_________________________________________________________________
dense_1 (Dense)              (None, 10, 2)             514       
Total params: 497,666
Trainable params: 497,666
Non-trainable params: 0
_________________________________________________________________
Train on 153 samples, validate on 17 samples
Epoch 1/10
153/153 [==============================] - 7s 48ms/step - loss: 0.3193 - acc: 0.9137 - val_loss: 0.0414 - val_acc: 1.0000
Epoch 2/10
153/153 [==============================] - 6s 42ms/step - loss: 0.2906 - acc: 0.9196 - val_loss: 0.1210 - val_acc: 1.0000
Epoch 3/10
153/153 [=====================